In [68]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load data
x_train = np.load('exam2_train_x.npy').astype(np.float32)
y_train = np.load('exam2_train_y.npy').astype(int)
x_train = x_train/255.
y_train = tf.one_hot(indices=y_train, depth=6)

x_test = np.load('exam2_test_x.npy').astype(np.float32)
y_test = np.load('exam2_test_y.npy').astype(int)
x_test = x_test/255.
y_test = tf.one_hot(indices=y_test, depth=6)

In [69]:
def one_hot_encoding(y, m):

    encoded_matrix = np.zeros((m, 6))

    for i in range(m):
        encoded_matrix[i, y[i]] = 1.0
      
    return encoded_matrix


In [70]:
def conv_model(features, mode, labels):
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 3])
    
    c1 = tf.layers.conv2d(inputs=input_layer, filters=8, kernel_size=[4, 4], padding='same',
                          activation=tf.nn.relu, strides=2)
    p1 = tf.layers.max_pooling2d(inputs=c1, pool_size=[5, 5], strides=1)
    
    c2 = tf.layers.conv2d(inputs=p1, filters=16, kernel_size=[4, 4], padding='valid',
                          activation=tf.nn.relu, strides=2)
    p2 = tf.layers.max_pooling2d(inputs=c2, pool_size=[5, 5], strides=1)
    
    a3 = tf.reshape(p2, [-1, 9*9*16])
    z3 = tf.layers.dense(a3, units=108, activation=tf.nn.relu)
    
    z4 = tf.layers.dense(z3, units=6, activation=tf.nn.sigmoid)

    predictions = {'classes': tf.arg_max(input=z4, axis=1),
                   'probabilities': tf.nn.softmax(z4, name="softmax_tensor")}
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=z4, labels=labels))
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [71]:
tf.set_random_seed(5)

alpha = 0.01
epochs = 1000

(m, nh, nw, nc) = x_train.shape

# n = y_train.shape[0]
# y_train = one_hot_encoding(y_train, n)
# n = y_test.shape[0]
# y_test = one_hot_encoding(y_test, n)
# 
# ny = y_train.shape[1]

# x = tf.placeholder(tf.float32, [None, nh, nw, nc])
# y = tf.placeholder(tf.float32, [None, ny])
# 
# z4 = forward(x_train)
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=z4, labels=y_train))
# 
# optimizer = tf.train.AdamOptimizer(learning_rate=alpha).minimize(cost)
# init = tf.global_variables_initializer()

clf = tf.estimator.Estimator(model_fn=conv_model)

log_hook = tf.train.LoggingTensorHook(tensors={"probabilities": "softmax_tensor"}, every_n_iter=100)

train_input = tf.estimator.inputs.numpy_input_fn(x={'x': x_train}, y=y_train, batch_size=85, 
                                   num_epochs=None, shuffle=True)
clf.train(input_fn=train_input, steps=1, hooks=[log_hook])
# with tf.Session() as session:
#     session.run(init)
#     
#     for epoch in range(epochs):
#         _, temp_cost = session.run([optimizer, cost], feed_dict={x: x_train})
# 
#     predict = tf.argmax(z4, 1)
#     correct = tf.equal(predict, tf.argmax(y_train, 1))
#     
#     accuracy = tf.reduce_mean(tf.cast(correct, "float"))
#     train_acc = accuracy.eval({x: x_train, y: y_train})
#     test_acc = accuracy.eval({x: x_test, y: y_test})
#     print(train_acc, test_acc)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\dearm\\AppData\\Local\\Temp\\tmp2761v20n', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000029A812ACC18>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


TypeError: unhashable type: 'Dimension'